### Installing Libraries for LangChain and Supporting Tools


In [ ]:
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install faiss-gpu
!pip install langchain_experimental
!pip install "langchain[docarray]"
!pip install pylcs
!pip3 install pypdf

### Importing Necessary Libraries


In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import  PyPDFLoader
from langchain.vectorstores import  FAISS
from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings 
from langchain.prompts import PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chains import LLMChain
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
from time import monotonic
from langchain_groq import ChatGroq
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from dotenv import load_dotenv





import textwrap
import os
import ast
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    answer_correctness,
    faithfulness,
    answer_relevancy,
    context_recall,
    answer_similarity
)

from helper_functions import num_tokens_from_string, replace_t_with_space, replace_double_lines_with_one_line, split_into_chapters, is_similarity_ratio_lower_than_th, analyse_metric_results

load_dotenv()

c:\Users\N7\Anaconda3\envs\openai-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

### Setting Preferred Encoding for PyPDF on Google Colab


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding # For using PyPDF on google colab 

### Setting OPENAI and GROQ API keys

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

### Defining Path to Harry Potter PDF


In [4]:
hp_pdf_path ="Harry_Potter_Book_1_The_Sorcerers_Stone.pdf"

### Splitting the PDF into Chapters and Preprocessing


In [5]:
chapters = split_into_chapters(hp_pdf_path) 
chapters = replace_t_with_space(chapters)
print(len(chapters))

17


### Defining Prompt Template for Summarization


In [6]:
summarization_prompt_template = """Write an extensive summary of about of the following:

{text}

SUMMARY:"""

summarization_prompt = PromptTemplate(template=summarization_prompt_template, input_variables=["text"])

### Defining Function to Create Chapter Summaries using LLMs


In [7]:
def create_chapter_summary(chapter):
    """
    Creates a summary of a chapter using a large language model (LLM).

    Args:
        chapter: A Document object representing the chapter to summarize.

    Returns:
        A Document object containing the summary of the chapter.
    """

    chapter_txt = chapter.page_content  # Extract chapter text
    model_name = "gpt-3.5-turbo-0125"  # Specify LLM model
    llm = ChatOpenAI(temperature=0, model_name=model_name)  # Create LLM instance
    gpt_35_turbo_max_tokens = 16000  # Maximum token limit for the LLM
    verbose = False  # Set to True for more detailed output

    # Calculate number of tokens in the chapter text
    num_tokens = num_tokens_from_string(chapter_txt, model_name)

    # Choose appropriate chain type based on token count
    if num_tokens < gpt_35_turbo_max_tokens:
        chain = load_summarize_chain(llm, chain_type="stuff", prompt=summarization_prompt, verbose=verbose)
    else:
        chain = load_summarize_chain(llm, chain_type="map_reduce", map_prompt=summarization_prompt, combine_prompt=summarization_prompt, verbose=verbose)

    start_time = monotonic()  # Start timer
    doc_chapter = Document(page_content=chapter_txt)  # Create Document object for chapter
    summary = chain.invoke([doc_chapter])  # Generate summary using the chain
    print(f"Chain type: {chain.__class__.__name__}")  # Print chain type
    print(f"Run time: {monotonic() - start_time}")  # Print execution time

    # Clean up summary text
    summary = replace_double_lines_with_one_line(summary["output_text"])

    # Create Document object for summary
    doc_summary = Document(page_content=summary, metadata=chapter.metadata)

    return doc_summary

### Generating Summaries for Each Chapter


In [8]:
chapter_summaries = []
for chapter in chapters:
    chapter_summaries.append(create_chapter_summary(chapter))

c:\Users\N7\Anaconda3\envs\openai-env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


Chain type: StuffDocumentsChain
Run time: 12.952999999979511
Chain type: StuffDocumentsChain
Run time: 10.734999999869615
Chain type: StuffDocumentsChain
Run time: 7.922000000020489
Chain type: StuffDocumentsChain
Run time: 6.405999999959022
Chain type: StuffDocumentsChain
Run time: 9.234000000171363
Chain type: StuffDocumentsChain
Run time: 9.343999999808148
Chain type: StuffDocumentsChain
Run time: 6.780999999959022
Chain type: StuffDocumentsChain
Run time: 9.125
Chain type: StuffDocumentsChain
Run time: 7.2350000001024455
Chain type: StuffDocumentsChain
Run time: 9.546999999787658
Chain type: StuffDocumentsChain
Run time: 7.765999999828637
Chain type: StuffDocumentsChain
Run time: 6.7030000002123415
Chain type: StuffDocumentsChain
Run time: 6.0
Chain type: StuffDocumentsChain
Run time: 9.781999999890104
Chain type: StuffDocumentsChain
Run time: 7.733999999938533
Chain type: StuffDocumentsChain
Run time: 10.218000000109896
Chain type: StuffDocumentsChain
Run time: 6.547000000020489


### Function to Encode a Book into a Vector Store using OpenAI Embeddings


In [9]:
def encode_book(path, chunk_size=1000, chunk_overlap=200):
    """
    Encodes a PDF book into a vector store using OpenAI embeddings.

    Args:
        path: The path to the PDF file.
        chunk_size: The desired size of each text chunk.
        chunk_overlap: The amount of overlap between consecutive chunks.

    Returns:
        A FAISS vector store containing the encoded book content.
    """

    # Load PDF documents
    loader = PyPDFLoader(path)
    documents = loader.load()

    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    texts = text_splitter.split_documents(documents)
    cleaned_texts = replace_t_with_space(texts)

    # Create embeddings and vector store
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(cleaned_texts, embeddings)

    return vectorstore

### Encoding Chapter Summaries into Vector Store


In [10]:
def encode_chapter_summaries(chapter_summaries):
    """
    Encodes a list of chapter summaries into a vector store using OpenAI embeddings.

    Args:
        chapter_summaries: A list of Document objects representing the chapter summaries.

    Returns:
        A FAISS vector store containing the encoded chapter summaries.
    """

    embeddings = OpenAIEmbeddings()  # Create OpenAI embeddings
    chapter_summaries_vectorstore = FAISS.from_documents(chapter_summaries, embeddings)  # Create vector store
    return chapter_summaries_vectorstore

### Creating Vector Stores and Retrievers for Book and Chapter Summaries


In [4]:
# ### IF VECTOR STORES ALREADY EXIST, LOAD THEM
if os.path.exists("chunks_vector_store") and os.path.exists("chapter_summaries_vector_store"):
    embeddings = OpenAIEmbeddings()
    chunks_vector_store =  FAISS.load_local("chunks_vector_store", embeddings, allow_dangerous_deserialization=True)
    chapter_summaries_vector_store =  FAISS.load_local("chapter_summaries_vector_store", embeddings, allow_dangerous_deserialization=True)

else:
    chunks_vector_store = encode_book(hp_pdf_path, chunk_size=1000, chunk_overlap=200)
    chapter_summaries_vector_store = encode_chapter_summaries(chapter_summaries)

    chunks_vector_store.save_local("chunks_vector_store") # save the chunks_vector_store
    chapter_summaries_vector_store.save_local("chapter_summaries_vector_store") # save the chapter_summaries_vector_store


c:\Users\N7\Anaconda3\envs\openai-env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


### Create retrievers from the vector stores

In [5]:
chunks_retriever = chunks_vector_store.as_retriever(search_kwargs={"k": 1})     
chapter_summaries_retriever = chapter_summaries_vector_store.as_retriever(search_kwargs={"k": 1})

### Agrregate retrieved content as string context

In [6]:
def create_chunks_context_per_question(question, chunks_query_retriever):
 
    # Retrieve relevant documents
    docs = chunks_query_retriever.get_relevant_documents(question)

    # Concatenate document content
    context = " ".join(doc.page_content for doc in docs)

    return context



def create_summaries_context_per_question(question, chapter_summaries_query_retriever):
   
    # Retrieve relevant chapter summaries
    docs_summaries = chapter_summaries_query_retriever.get_relevant_documents(question)


    # Concatenate chapter summaries with citation information
    context_summaries = "".join(
        f"{doc.page_content} (Chapter {doc.metadata['chapter']})" for doc in docs_summaries
    )

    return context_summaries


### LLM based function to distill only relevant retrieved content

In [7]:
keep_only_relevant_content_prompt_template = """you receive a query: {query} and retrieved docuemnts: {retrieved_documents} from a vector store.
 You need to filter the retrieved data and keep only the sentences that are relevant, but all of them.
 you should output the distilled content in a json format. 
 REMEMBER: the output has to be a json containing ALL the relevant sentences, and not the answer to the query. {format_instructions}"""

class QuestionAnswer(BaseModel):
    relevant_content: str = Field(description="The relevant content from the retrieved documents that is relevant to the query.")


json_parser = JsonOutputParser(pydantic_object=QuestionAnswer)

keep_only_relevant_content_prompt = PromptTemplate(
    template=keep_only_relevant_content_prompt_template,
    input_variables=["query", "retrieved_documents"],
    partial_variables={"format_instructions": json_parser.get_format_instructions()}, 
)

keep_only_relevant_content_llm = ChatGroq(temperature=0, model_name="llama3-70b-8192", groq_api_key=groq_api_key, max_tokens=4000)
keep_only_relevant_content_chain = keep_only_relevant_content_prompt | keep_only_relevant_content_llm | json_parser

def keep_only_relevant_content(question, context, chain):
    """
    Keeps only the relevant content from the retrieved documents that is relevant to the query.

    Args:
        question: The query question.
        context: The retrieved documents.
        chain: The LLMChain instance.

    Returns:
        The relevant content from the retrieved documents that is relevant to the query.
    """

    # Create Document objects for the query and retrieved documents
    doc_query = Document(page_content=question)
    doc_retrieved_documents = Document(page_content=context)

    input_data = {
    "query": doc_query,
    "retrieved_documents": doc_retrieved_documents
}
    # Invoke the chain to keep only the relevant content
    output = chain.invoke(input_data)

    return output

### combine retrival with content distilation 

In [11]:
def get_relevant_chunks_per_question(question, chunks_retriever, keep_only_relevant_content_chain):
    """
    Retrieves relevant chunks of text from the book based on a question.

    Args:
        question: The question to ask.
        chunks_retriever: The retriever for the book chunks.
        keep_only_relevant_content_chain: The chain to keep only the relevant content.

    Returns:
        The relevant chunks of text from the book based on the question.
    """

    # Get the context for the question
    context = create_chunks_context_per_question(question, chunks_retriever)

    # Keep only the relevant content from the retrieved documents that is relevant to the query
    relevant_content = keep_only_relevant_content(question, context, keep_only_relevant_content_chain)

    return relevant_content


def get_relevant_summaries_per_question(question, chapter_summaries_retriever, keep_only_relevant_content_chain):
    """
    Retrieves relevant chapter summaries based on a question.

    Args:
        question: The question to ask.
        chapter_summaries_retriever: The retriever for the chapter summaries.
        keep_only_relevant_content_chain: The chain to keep only the relevant content.

    Returns:
        The relevant chapter summaries based on the question.
    """

    # Get the context for the question
    context_summaries = create_summaries_context_per_question(question, chapter_summaries_retriever)

    # Keep only the relevant content from the retrieved documents that is relevant to the query
    relevant_content_summaries = keep_only_relevant_content(question, context_summaries, keep_only_relevant_content_chain)

    return relevant_content_summaries


### LLM based function to determine if retrieved content is relevant to question

In [12]:
is_relevant_content_prompt_template = """you receive a query: {query} and a document: {document} from a vector store. 
You need to determine if the document is relevant to the query. {format_instructions}"""

class Relevance(BaseModel):
    is_relevant: bool = Field(description="Whether the document is relevant to the query.")

json_parser = JsonOutputParser(pydantic_object=Relevance)
is_relevant_llm = ChatGroq(temperature=0, model_name="llama3-70b-8192", groq_api_key=groq_api_key, max_tokens=4000)

is_relevant_content_prompt = PromptTemplate(
    template=is_relevant_content_prompt_template,
    input_variables=["query", "document"],
    partial_variables={"format_instructions": json_parser.get_format_instructions()},
)
is_relevant_content_chain = is_relevant_content_prompt | is_relevant_llm | json_parser

def is_relevant_content(question, document, chain):
    """
    Determines if a document is relevant to a query.

    Args:
        question: The query question.
        document: The document to evaluate.
        chain: The LLMChain instance.

    Returns:
        Whether the document is relevant to the query.
    """

    # Create Document objects for the query and document
    doc_query = Document(page_content=question)
    doc_document = Document(page_content=document)

    input_data = {
    "query": doc_query,
    "document": doc_document
}

    # Invoke the chain to determine if the document is relevant
    output = chain.invoke(input_data)

    return output

### LLM based function to re-write a question

In [9]:
### Question Re-writer

class RewriteQuestion(BaseModel):
    """
    Output schema for the rewritten question.
    """
    rewritten_question: str = Field(description="The improved question optimized for vectorstore retrieval.")

rewrite_question_string_parser = JsonOutputParser(pydantic_object=RewriteQuestion)


rewrite_llm = ChatGroq(temperature=0, model_name="llama3-70b-8192", groq_api_key=groq_api_key, max_tokens=4000)
rewrite_prompt_template = """You are a question re-writer that converts an input question to a better version optimized for vectorstore retrieval.
 Analyze the input question {question} and try to reason about the underlying semantic intent / meaning.
 {format_instructions}
 """

# Prompt

rewrite_prompt = PromptTemplate(
    template=rewrite_prompt_template,
    input_variables=["question"],
    partial_variables={"format_instructions": rewrite_question_string_parser.get_format_instructions()},
)



question_rewriter = rewrite_prompt | rewrite_llm | rewrite_question_string_parser  # Combine prompt, LLM, and parser

def rewrite_question(question):
    """Rewrites the given question using the LLM."""
    result = question_rewriter.invoke({"question": question})
    return result 



### LLM based function to answer a question given context

In [ ]:
class QuestionAnswerFromContext(BaseModel):
    relevant_content: str = Field(description="answer a question from a given context.")

question_answer_from_context_json_parser = JsonOutputParser(pydantic_object=QuestionAnswerFromContext)
question_answer_from_context_llm = ChatGroq(temperature=0, model_name="llama3-70b-8192", groq_api_key=groq_api_key, max_tokens=4000)
question_answer_from_context_prompt_template = """you receive a query: {query} and a context: {context} from a vector store. 
You need to answer the query from the context. {format_instructions}"""

question_answer_from_context_prompt = PromptTemplate(
    template=question_answer_from_context_prompt_template,
    input_variables=["query", "context"],
    partial_variables={"format_instructions": question_answer_from_context_json_parser.get_format_instructions()},
)
question_answer_from_context_chain = question_answer_from_context_prompt | question_answer_from_context_llm | question_answer_from_context_json_parser

def answer_question_from_context(question, context, chain):
    """
    Answers a question from a given context.

    Args:
        question: The query question.
        context: The context to answer the question from.
        chain: The LLMChain instance.

    Returns:
        The answer to the question from the context.
    """

    # Create Document objects for the query and context
    doc_query = Document(page_content=question)
    doc_context = Document(page_content=context)

    input_data = {
    "query": doc_query,
    "context": doc_context
}

    # Invoke the chain to answer the question from the context
    output = chain.invoke(input_data)

    return output

### LLM based function to check if an answer is hallucination

In [10]:
class is_hallucination(BaseModel):
    """
    Output schema for the rewritten question.
    """
    is_hallucination: bool = Field(description="Answer is grounded in the facts, 'yes' or 'no'")
hallucination_parser = JsonOutputParser(pydantic_object=is_hallucination)
is_hallucination_llm = ChatGroq(temperature=0, model_name="llama3-70b-8192", groq_api_key=groq_api_key, max_tokens=4000)
is_hallucination_prompt_template = """You are a fact-checker that determines if the answer to the question is grounded in the facts.
 Analyze the input question {question} and the answer {answer} and determine if the answer is grounded in the facts.
 {format_instructions}
 """
is_hallucination_prompt = PromptTemplate(
    template=is_hallucination_prompt_template,
    input_variables=["question", "answer"],
    partial_variables={"format_instructions": hallucination_parser.get_format_instructions()},
)
is_hallucination_chain = is_hallucination_prompt | is_hallucination_llm | hallucination_parser

def is_answer_hallucination(question, answer):
    """Determines if the answer to the question is grounded in the facts."""
    result = is_hallucination_chain.invoke({"question": question, "answer": answer})
    return result


### LLM based function to determine if a question can be fully answered given a context

In [8]:
can_be_answered_prompt_template = """You receive a query: {question} and a context: {context}. 
You need to determine if the question can be fully answered based on the context.
{format_instructions}

**Answer:**
"""

class QuestionAnswer(BaseModel):
    can_be_answered: bool = Field(description="binary result of whether the question can be fully answered or not")

can_be_answered_json_parser = JsonOutputParser(pydantic_object=QuestionAnswer)

answer_question_prompt = PromptTemplate(
    template=can_be_answered_prompt_template,
    input_variables=["question","context"],
    partial_variables={"format_instructions": can_be_answered_json_parser.get_format_instructions()},
)

can_be_answered_llm = ChatGroq(temperature=0, model_name="llama3-70b-8192", groq_api_key=groq_api_key, max_tokens=4000)
can_be_answered_chain = answer_question_prompt | can_be_answered_llm | can_be_answered_json_parser


def can_be_answered_from_contenxt(question, context, can_be_answered_chain):
    """
    Determines if a question can be answered based on the context.

    Args:
        question: The query question.
        context: The retrieved documents.
        chain: The LLMChain instance.

    Returns:
        A binary result of whether the question can be answered or not.
    """

    # Create Document objects for the query and context
    doc_question = Document(page_content=question)
    doc_context = Document(page_content=context)

    input_data = {
        "question": doc_question,
        "context": doc_context
    }

    # Invoke the chain to determine if the question can be answered
    output = can_be_answered_chain.invoke(input_data)

    return output
    

In [12]:
from langchain.agents import initialize_agent, Tool
from langchain.chains.question_answering import load_qa_chain
from langchain.memory import ConversationBufferWindowMemory, ConversationSummaryMemory


llm_retrieval_agent = ChatGroq(temperature=0, model_name="llama3-70b-8192", groq_api_key=groq_api_key, max_tokens=4000)


# # Craft agent description emphasizing reliance on retrieval
agent_description = """
You are an AI assistant tasked with answering questions about the content of a book which is encoded into vector stores. you have both chunks encoded and chapter summaries encoded.
You have no prior knowledge of any book, including characters, places, or events. 
Your answers must be based solely on the information you retrieve using the provided tools.
You can use the RetrieveBookContent tool to search for specific details within the book content, and the Retrievechaptersummary tool to get a high-level overview of events and key points from the chapter summaries.
every answer you provide should be supported by a *CITED* evidence from the book content or chapter summaries.
if no relevant information is found, you should indicate that you were unable to find any relevant information.
you must also determine if a question can be answered based on the context provided.
"""


tools = [
Tool(
    name="RetrieveBookContent",
    func=lambda question: get_relevant_chunks_per_question(question, chunks_retriever, keep_only_relevant_content_chain),
    description="Retrieves relevant chunks of text from the book based on a question.",
),
Tool(
    name="RetrieveChapterSummary",
    func=lambda question: get_relevant_summaries_per_question(question, chapter_summaries_retriever, keep_only_relevant_content_chain),
    description="Retrieves relevant chapter summaries based on a question.",
),
# Tool(
#     name="CanBeAnswered",
#     func=lambda question, context: can_be_answered_from_contenxt(question, context, can_be_answered_chain),
#     description="Determines if a question can be answered based on the context.",
# ),
]


# Initialize QA Chain and Memory
chain = load_qa_chain(llm_retrieval_agent, chain_type="stuff", verbose=True)  # Properly initialize the QA chain
# memory = ConversationBufferWindowMemory(k=2, return_messages=True)

# Initialize Agent with QA Chain
agent = initialize_agent(
    tools, 
    llm_retrieval_agent, 
    agent="zero-shot-react-description", 
    verbose=True,
    agent_description=agent_description,
    # max_iterations=7,
    # memory=memory,
    handle_parsing_errors=True,
    chain=chain,  # Include the QA chain in agent initialization
)



c:\Users\N7\Anaconda3\envs\openai-env\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [14]:
query = "how did harry beat quirrell?"
result = agent.invoke(query)




> Entering new AgentExecutor chain...
Thought: I need to find out how Harry beat Quirrell in the first book.

Action: RetrieveBookContent
Action Input: "Harry" and "Quirrell" and "final battle"

Observation: {'relevant_content': 'Harry went on feverishly, “then Voldemort will be able to come and finish me off ... Well, I suppose Bane’ll be happy.”'}
Thought:Thought: It seems like the retrieved content is not directly answering my question. I need to find more information about the final battle between Harry and Quirrell.

Action: RetrieveBookContent
Action Input: "Harry" and "Quirrell" and "final battle" and "fire"

Observation: {'relevant_content': ['A lamp flickered on. It was Hermione Granger, wearing a pink bathrobe and a frown.', 'Harry couldn’t believe anyone could be so interfering.', '“You!” said Ron furiously. “Go back to bed!”', '“I almost told your brother,” Hermione snapped, “Percy — he’s a prefect, he’d put a stop to this.”', 'Harry couldn’t believe anyone could be so in

### Generating Sub-Questions for Vector Store Queries from User Questions


In [ ]:
modifier_prompt_template = """
You are an expert on analyzing and understanding books based solely on their textual content. You have no prior knowledge about the specific book in the question.
If you ever read the related book before, FORGET ANYTHING about it.
You have access to two vector stores:

• One containing all the book content divided into chunks of 1000 characters with 200 character overlap.
• Another containing summaries of each chapter, approximately 250 tokens each.

Given a user's question about the book, your task is to generate a list of no more than 3 sub-questions that can be used as queries to retrieve relevant information from the vector stores based on semantic similarity. These sub-questions should be designed to collectively cover all the information needed to answer the original question comprehensively, without relying on any prior knowledge of the book's plot, characters, or events.

When generating the sub-questions, consider the following:


1.No Pre-knowledge: you're unaware of the book's details like plot or characters. The sub-questions must not present any prior knowledge of the book.
2.Directly Derived: Create sub-questions strictly from the user's question, aiming to retrieve book information without presupposing specific plot details or events.
3.Break Down: Decompose the user's question into finer, detailed sub-questions using only the textual content provided.
4.Key Concepts: Identify essential information needed from the original question and form targeted sub-questions to gather this data.
5.Self-contained Queries: Each sub-question should stand alone for effective vector store querying through semantic similarity.
6.Logical Sequence: Arrange sub-questions in a logical order that collectively provides a thorough answer.
7.Efficiency: Ensure sub-questions are unique and focused to avoid redundant searches and streamline information retrieval.

Output your response as a Python list, where each item is a self-contained sub-question that can be used as a standalone query for vector retrieval.

User's question: {question}
"""
modifier_prompt = PromptTemplate(
input_variables=["question"],
template=modifier_prompt_template,
)

llm = ChatOpenAI(temperature=0, model_name="gpt-4-1106-preview", max_tokens=4000)

question_modifier_llm_chain = LLMChain(llm=llm, prompt=modifier_prompt)

### Modifying and Cleaning User Questions for Enhanced Querying


In [ ]:
def modify_question(question):
    """
    Modifies a question using a question modifier LLM chain and cleans up the output.

    Args:
        question: The input question string.

    Returns:
        A list of modified questions, or an empty list if there's an error.
    """

    # Invoke the question modifier LLM chain
    result = question_modifier_llm_chain.invoke(input=question)
    modified_questions_str = result['text']

    # Clean up the output string
    clean_str = modified_questions_str.replace("```python", "").replace("```", "").strip()  # Remove Markdown code block syntax
    clean_str = clean_str.replace("sub_questions = ", "").strip()  # Remove variable assignment
    clean_str = textwrap.dedent(clean_str)  # Dedent to remove unexpected indents

    # Attempt to convert the cleaned string into a list of questions
    try:
        modified_questions = ast.literal_eval(clean_str)
    except SyntaxError as e:
        print(f"Syntax error during ast.literal_eval: {e}")
        modified_questions = []  # Default to an empty list in case of error

    return modified_questions

### Example: Generating Modified Questions from an Original Query


In [ ]:
question = "how did harry beat quirrell?"
modified_question = modify_question(question)
print(modified_question) # watch new questions generated based on the original question

### Creating Context for a Question by Retrieving Relevant Documents and Summaries


In [ ]:
def create_context_per_question(question, multi_query_retriever, multi_query_retriever_chapter_summaries):
    """
    Creates context for a question by retrieving relevant documents and chapter summaries.

    Args:
        question: The input question string.
        multi_query_retriever: A retriever for retrieving relevant documents.
        multi_query_retriever_chapter_summaries: A retriever for retrieving relevant chapter summaries.

    Returns:
        A tuple containing two strings:
            - context: The concatenated content of relevant documents.
            - context_summaries: The concatenated content of relevant chapter summaries with citation information.
    """

    # Retrieve relevant documents and chapter summaries
    docs = multi_query_retriever.get_relevant_documents(question)
    docs_summaries = multi_query_retriever_chapter_summaries.get_relevant_documents(question)

    # Concatenate document content
    context = " ".join(doc.page_content for doc in docs)

    # Concatenate chapter summaries with citation information
    context_summaries = "".join(
        f"{doc.page_content} (Chapter {doc.metadata['chapter']})" for doc in docs_summaries
    )

    return context, context_summaries

### Comprehensive Question Answering Pipeline Using Context Retrieval and LLM


In [ ]:
def answer_question_pipeline(question, chunks_retriever, chapter_summaries_retriever, answer_from_context_llm_chain, multi_query_retriver_llm, similarity_th=0.5):
    """
    Answers a question by retrieving relevant context, modifying the question, and using an LLM chain.

    Args:
        question: The input question string.
        retriever: A retriever for retrieving relevant documents.
        chapter_summaries_retriever: A retriever for retrieving relevant chapter summaries.
        answer_from_context_llm_chain: An LLM chain for answering questions based on context.
        multi_query_retriver_llm: An LLM for use in the MultiQueryRetriever.
        similarity_th:  Similarity threshold for context accumulation

    Returns:
        A tuple containing:
            - result: The result of invoking the answer_from_context_llm_chain.
            - all_context_book: The concatenated content of relevant documents.
            - all_context_summaries: The concatenated content of relevant chapter summaries.
    """

    # Create MultiQueryRetrievers
    multi_query_retriever = MultiQueryRetriever.from_llm(retriever=chunks_retriever, llm=multi_query_retriver_llm)
    multi_query_retriever_chapter_summaries = MultiQueryRetriever.from_llm(retriever=chapter_summaries_retriever, llm=multi_query_retriver_llm)

    # Modify the question
    modified_questions = modify_question(question)

    # Accumulate relevant context
    all_context_book = ""
    all_context_summaries = ""

    for modified_question in modified_questions:
        curr_question_relevant_context, curr_question_relevant_summaries_context = create_context_per_question(
            modified_question, multi_query_retriever, multi_query_retriever_chapter_summaries
        )

        # Add context if it's not too similar to existing context
        if is_similarity_ratio_lower_than_th(all_context_book, curr_question_relevant_context, similarity_th):
            all_context_book += curr_question_relevant_context

        if is_similarity_ratio_lower_than_th(all_context_summaries, curr_question_relevant_summaries_context, similarity_th):
            all_context_summaries += curr_question_relevant_summaries_context

    # Combine context from book and summaries
    all_context = all_context_book + all_context_summaries

    # Prepare input data for the LLM chain
    input_data = {
        "context": all_context,
        "question": question
    }

    # Execute the LLM chain and get the response
    result = answer_from_context_llm_chain.invoke(input=input_data)

    return result, all_context_book, all_context_summaries


### Template for Answering Questions Using Context-Specific Information


In [ ]:
answer_question_prompt_template = """
Based solely on the information provided in this context, and without using any information outside of this context, please answer the following question as concisely and as shortly as possible. You can rephrase the question for better fitting to the context.

Context:{context}
Question:{question}

**If the answer cannot be derived from the context, or if it requires knowledge from outside sources, simply answer: "I don't know".**

Please cite specific parts of the context in your answer to demonstrate how it supports your response.
If the chapter number of the relevant context appears, specify it in your answer.
"""
answer_question_prompt = PromptTemplate(
input_variables=["context", "question"],
template=answer_question_prompt_template,
)

multi_query_retriver_llm = ChatOpenAI(temperature=0, model_name="gpt-4-1106-preview", max_tokens=4000)

answer_from_context_llm_chain = LLMChain(llm=llm, prompt=answer_question_prompt)

### Example Execution of the Question Answering Pipeline for "How Did Harry Beat Quirrell?"


In [ ]:
question = 'how did harry beat quirrell?'
result, all_context_book, all_context_summaries = answer_question_pipeline(question,chunks_retriever,chapter_summaries_retriever, answer_from_context_llm_chain, multi_query_retriver_llm)

wrapped_all_context_book = textwrap.fill(all_context_book, width=120)
print(f' conetxt book: {wrapped_all_context_book} \n')

wrapped_all_context_summaries = textwrap.fill(all_context_summaries, width=120)
print(f' context summaries: {wrapped_all_context_summaries} \n')

wrapped_result = textwrap.fill(result['text'], width=120)
print(f' answer: {wrapped_result}')

### Model Evaluation


In [ ]:
questions = [
    "Who gave Harry Potter his first broomstick?",
    "What is the name of the three-headed dog guarding the Sorcerer's Stone?",
    "Which house did the Sorting Hat initially consider for Harry?",
    "What is the name of Harry's owl?"
]
#     "How did Harry and his friends get past Fluffy?",
#     "What is the Mirror of Erised?",
#     "Who tried to steal the Sorcerer's Stone?",
#     "How did Harry defeat Quirrell/Voldemort?",
#     "What is Harry's parent's secret weapon against Voldemort?",
# ]

ground_truth_answers = [
    "Professor McGonagall",
    "Fluffy",
    "Slytherin",
    "Hedwig",
    # "They played music to put Fluffy to sleep.",
    # "A magical mirror that shows the 'deepest, most desperate desire of our hearts.'",
    # "Professor Quirrell, possessed by Voldemort",
    # "Harry's mother's love protected him, causing Quirrell/Voldemort pain when they touched him.",
    # "Love",
]

### Generating Answers and Retrieving Documents for Predefined Questions


In [ ]:
generated_answers = []
retrieved_documents = []
for question in questions:
    result, all_context_book, all_context_summaries = answer_question_pipeline(question, chunks_retriever, chapter_summaries_retriever, answer_from_context_llm_chain, multi_query_retriver_llm)
    generated_answers.append(result['text'])
    retrieved_documents.append(all_context_book + all_context_summaries)


### Displaying Retrieved Documents and Generated Answers


In [ ]:
print(f'retrieved_documents: {retrieved_documents}\n')
print(f'generated_answers: {generated_answers}')

### Preparing Data and Conducting Ragas Evaluation


In [ ]:
# Prepare data for Ragas evaluation
data_samples = {
    'question': questions,  # Replace with your list of questions
    'answer': generated_answers,  # Replace with your list of generated answers
    'contexts': retrieved_documents,  # Your retrieved_documents list
    'ground_truth': ground_truth_answers  # Replace with your list of ground truth answers
}

# Convert contexts to list of strings (if necessary)
data_samples['contexts'] = [list(context) for context in data_samples['contexts']]

dataset = Dataset.from_dict(data_samples)

# Evaluate using Ragas with the specified metrics
metrics = [
    answer_correctness,
    faithfulness,
    answer_relevancy,
    context_recall,
    answer_similarity
]
llm = ChatOpenAI(temperature=0, model_name="gpt-4-1106-preview", max_tokens=4000)
score = evaluate(dataset, metrics=metrics, llm=llm)

# Print results and explanations
results_df = score.to_pandas()
print(results_df)

### Analyzing Metric Results from Ragas Evaluation


In [ ]:
analyse_metric_results(results_df)

### Interactive Chat Interface for Harry Potter Inquiries


In [ ]:
def chat_with_data(chunks_retriever, chapter_summaries_retriever, answer_from_context_llm_chain, multi_query_retriver_llm):
    """
    Provides an interactive chat interface for answering questions about Harry Potter.

    Args:
        retriever: A retriever for retrieving relevant documents.
        chapter_summaries_retriever: A retriever for retrieving relevant chapter summaries.
        answer_from_context_llm_chain: An LLM chain for answering questions based on context.
        multi_query_retriver_llm: An LLM for use in the MultiQueryRetriever.
    """

    print("You can start chatting with me about Harry Potter. Type 'exit' to stop.")

    while True:
        # Prompt the user for a question
        question = input("What's your question? \n")

        # Check if the user wants to exit
        if question.lower() == 'exit':
            print("Exiting chat. Goodbye!")
            break

        # Answer the question using the pipeline
        result, _, _ = answer_question_pipeline(
            question, chunks_retriever, chapter_summaries_retriever, answer_from_context_llm_chain, multi_query_retriver_llm
        )

        # Print the answer
        print("Answer:")
        wrapped_result = textwrap.fill(result['text'], width=120)  # Wrap text for readability
        print(wrapped_result)
        print("-" * 80)  # Print a separator line for readability

### Calling the chat_with_data function

In [ ]:
chat_with_data(chunks_retriever,chapter_summaries_retriever, answer_from_context_llm_chain, multi_query_retriver_llm)